In [47]:
#Data extraction from .raw file and plot graphs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from numpy_ext import rolling_apply
import datetime
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.pyplot import figure
import voila
import io

w = widgets.IntSlider()
uploader = widgets.FileUpload(accept='.raw', multiple=False)
display(uploader)

run_button = widgets.Button(description='Run analysis')
out = widgets.Output()

def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    slope = slope/6
    return slope

def getSec(s):
    datee = datetime.datetime.strptime(s, "%H%M%S")
    return datee.hour * 3600 + datee.minute * 60 + datee.second

def run(_):
    with out:
        print("Working...")
        print("")
        print("")
        print("")
        print("")
        input_file = list(uploader.value.values())[0]
        content = input_file['content']
        content = io.StringIO(content.decode('utf-8'))
        df = pd.read_csv(content, sep=" ", header = None)
        df = df.iloc[: , 4:]
        df = df.fillna(0)
        dataframe = df.astype(str) 
        dataframe = dataframe[~dataframe[5].str.contains("?", regex=False)]
        dataframe = dataframe.astype(float)
        dataframe = dataframe.astype(int)
        dataframe = dataframe.where(dataframe > 9999, np.nan)
        dataframe_final = dataframe.apply(lambda x: pd.Series(x.dropna().to_numpy()), axis=1)
        dataframe_final = dataframe_final.set_axis(dataframe.columns[:dataframe_final.shape[1]], axis=1).reindex(dataframe.columns, axis=1)
        dataframe_final.dropna(axis=1, how='all', inplace = True)
        dataframe_final[dataframe_final.columns[1]] = dataframe_final[dataframe_final.columns.values[1]]/10
        dataframe_final.columns = ["Time", "Magnetic_Readings"]

        dataframe_final["Time"] = dataframe_final["Time"].astype(str)
        for i in range(len(dataframe_final)):
            dataframe_final["Time"].values[i] = dataframe_final["Time"].values[i][:-2]
            dataframe_final["Time"].values[i] = getSec(dataframe_final["Time"].values[i])

        dataframe_final["Time"] = dataframe_final["Time"].astype(int)
        df_time = pd.read_csv("Desktop\Time.csv")
        df_merged = pd.merge(df_time, dataframe_final, how='left')

        dataframe_final["Gradients"] = abs((dataframe_final["Magnetic_Readings"].rolling(100, min_periods=100).apply(calc_slope))*600)
        df_merged_slopes = pd.merge(df_merged, dataframe_final, how='left')
        plt.figure(figsize=(24,6))
        plt.scatter(df_merged_slopes["Time"], df_merged_slopes["Gradients"], 1, "black")
        plt.xlabel("Time (sec)")
        plt.ylabel("Gradient (nT/10 min)")
        plt.title("Gradients vs. Time")
        plt.axhline(y=10, color='r', linestyle='-', label="Threshold: 10 nt/10 min")
        plt.legend(loc = 'upper left')
        plt.show()
        if (df_merged_slopes["Gradients"] > 10).any() is True:
            print("Gradient change > 10 nT/10 min found; must be re-flown.")
            print("")
            print("")
            print("")
            print("")
        else:
            print("No aberrant gradients recorded.")
            print("")
            print("")
            print("")
            print("")

        df_merged["600s Moving Average"] = df_merged['Magnetic_Readings'].rolling(600, min_periods=2).mean() #Troubleshoot so misconstrued means aren't calculated!
        df_merged["Variation From 600s Moving Average"] = abs(df_merged['Magnetic_Readings'] - df_merged["600s Moving Average"])
        plt.figure(figsize=(24,6))
        plt.scatter(df_merged["Time"], df_merged["Variation From 600s Moving Average"], 1, "black")
        plt.xlabel("Time (sec)")
        plt.ylabel("Variation From 600s Moving Average (nT)")
        plt.title("Variation From 600s Moving Average vs. Time")
        plt.axhline(y=10, color='r', linestyle='-', label="Threshold: 10 nt")
        plt.legend(loc = 'upper left')
        plt.show()
        if (df_merged["Variation From 600s Moving Average"] > 10).any() is True:
            print("Variation from 600s moving average > 10 nT found; must be re-flown.")
            print("")
            print("")
            print("")
            print("")
        else:
            print("No aberrant variations from 600s moving average recorded.")
            print("")
            print("")
            print("")
            print("")

        aberrant = pd.DataFrame(columns=df_merged_slopes.columns)
        cond = df_merged["Variation From 600s Moving Average"] > 10
        rows = df_merged.loc[cond, :]
        aberrant = pd.concat([aberrant, rows], ignore_index=True)
        cond = df_merged_slopes["Gradients"] > 10
        rows = df_merged.loc[cond, :]
        aberrant = pd.concat([aberrant, rows], ignore_index=True)

        if (df_merged_slopes["Gradients"] > 10).any() is True | (df_merged["Variation From 600s Moving Average"] > 10).any() is True:
            plt.figure(figsize=(24,6))
            plt.scatter(aberrant["Time"], aberrant["Magnetic_Readings"], 1, "red", label="aberrant")
            plt.scatter(df_merged["Time"], df_merged["Magnetic_Readings"], 1, "black", label="normal")
            plt.xlabel("Time (sec)")
            plt.ylabel("Magnetic Readings (nT)")
            plt.title("Magnetic Readings vs. Time")
            plt.show()
        else:
            plt.figure(figsize=(24,6))
            plt.scatter(df_merged["Time"], df_merged["Magnetic_Readings"], 1, "black")
            plt.xlabel("Time (sec)")
            plt.ylabel("Magnetic Readings (nT)")
            plt.title("Magnetic Readings vs. Time")
            plt.show()

display(run_button, out)
run_button.on_click(run)

FileUpload(value={}, accept='.raw', description='Upload')

Button(description='Run analysis', style=ButtonStyle())

Output()